In [ ]:
import src.models.difflib_bigrams as db
import os
import pandas as pd
from nltk.corpus import stopwords
from nltk import stem
import nltk
import xml.etree.ElementTree as ET
import string
import re
import numpy as np
import requests
import json
import glob
import time

from src.etl.get_anames import scrape_anames, retrieve_anames

In [ ]:
xmls_base = "src/data/temp/wiki_xmls/"
xmls_list = [x for x in os.listdir(xmls_base) if ".xml" in x]

resdict = {}
for fn in xmls_list:
    print(fn)
    
    # This block is for fixing broken xmls with no closing tags
    try:
        tree = ET.parse(xmls_base + fn)
    except Exception as e:
        with open(xmls_base + fn, "a") as app:
            app.write("  </page>")
            app.write("</mediawiki>")
        tree = ET.parse(xmls_base + fn)
        
    # Set up the tree and the list of results for the current article
    root = tree.getroot().find("{http://www.mediawiki.org/xml/export-0.10/}page")
    revlist = []

    for rev in root.findall("{http://www.mediawiki.org/xml/export-0.10/}revision"):
        # The dictionary for each revision
        curdict = {}

        curdict["time"] = rev.find("{http://www.mediawiki.org/xml/export-0.10/}timestamp").text
        txt = rev.find("{http://www.mediawiki.org/xml/export-0.10/}text").text
        
        if not txt is None:
            curdict["text"] = txt
        else:
            curdict["text"] = ""
            
        comm = rev.find("{http://www.mediawiki.org/xml/export-0.10/}comment")
        if not comm is None:
            curdict["comm"] = comm.text
        else:
            curdict["comm"] = ""

        cont = rev.find("{http://www.mediawiki.org/xml/export-0.10/}contributor")
        user = cont.find("{http://www.mediawiki.org/xml/export-0.10/}username")
        if not user is None:
            curdict["user"] = user.text
        else:
            curdict["user"] = cont.find("{http://www.mediawiki.org/xml/export-0.10/}ip").text

        revlist.append(curdict)
        
    resdict[fn[:-4]] = revlist

In [ ]:
for name, revl in resdict.items():
    print(name)
    print(revl[0]["time"],revl[-1]["time"])
    print(len(revl))
    print("________________")
        

In [ ]:
test = False

if test:
    pp_dir = "test/partisan_phrases/"
else:
    pp_dir = "src/data/init/partisan_phrases/"
    
pp_txts = os.listdir(pp_dir)
score_dict = {}
for i in pp_txts:
    with open(pp_dir + i) as curtxt:
        print(len(curtxt.readlines()))
        for line in curtxt.readlines()[1:]:
            splt = line.split("|")
            score_dict[splt[0]] = float(splt[1].strip())


In [ ]:
len(score_dict)

In [ ]:



cnt = 0

# Populate resdict with stats
for name, revl in resdict.items():
    prevr = db.bigram(preproc_strn(revl[0]["text"])[0])
    for rev in revl:
        if cnt % 1000 == 1:
            print(cnt)
        cnt += 1
        curr = db.bigram(preproc_strn(rev["text"])[0])
        diffs = db.unique_items(prevr, curr)
        rem, add = diffs
        
        # Trying to get the following output: [absscore, sumscore, numwords, counts_list, totphrs]
        rem_abs = 0
        add_abs = 0
        rem_sum = 0
        add_sum = 0
        rem_num = len(rem)
        add_num = len(add)
#         add_counts = {}
#         rem_counts = {}
        add_phrs = 0
        rem_phrs = 0
        
        for bigr in rem:
            if bigr in score_dict.keys():
                rem_abs += abs(score_dict[bigr])
                rem_sum += score_dict[bigr]
                rem_phrs += 1
                
        for bigr in add:  
            if bigr in score_dict.keys():
                add_abs += abs(score_dict[bigr])
                add_sum += score_dict[bigr]
                add_phrs += 1
        
        rev["rem"] = rem
        rev["add"] = add
        rev["rem_abs"] = rem_abs
        rev["add_abs"] = add_abs
        rev["rem_sum"] = rem_sum
        rev["add_sum"] = add_sum
        rev["rem_num"] = rem_num
        rev["add_num"] = add_num
        rev["rem_phrs"] = rem_phrs
        rev["add_phrs"] = add_phrs
        
        del rev["text"]
        prevr = curr
            

# [absscore, sumscore, numwords, counts_list, totphrs]

In [ ]:
biascnt = 0
cnt = 0
for name, revl in resdict.items():
    for rev in revl:
        cnt += 1
        if "bias" or "ideol" in rev["comm"].lower():
            print(rev["comm"])
            biascnt += 1
biascnt / cnt

In [ ]:
import json
rd_base = "src/data/temp/resdicts/"
with open(rd_base + "rd.json", "w") as outfile:  
    json.dump(resdict, outfile) 